In [1]:
import tensorflow as tf
import spektral
from spektral import transforms as T
from spektral import layers
import networkx as nx
from sklearn.model_selection import train_test_split
import math
import time

2024-02-18 20:16:19.461150: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-18 20:16:20.340248: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 20:16:20.340336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 20:16:20.479943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 20:16:20.608401: I tensorflow/core/platform/cpu_feature_guar

In [10]:
import tensorflow as tf

def resize_tensor(t, h, w):
    # Get the original shape of the input tensor
    original_shape = tf.shape(t)

    # Compute the dimensions for slicing and padding
    sliced_shape = tf.minimum(original_shape, [original_shape, h, w])
    padding = tf.maximum([original_shape[0]] - original_shape,h, w, 0)

    # Slice the input tensor to the desired shape
    sliced_tensor = tf.slice(t, begin=[0, 0, 0], size=sliced_shape)

    # Pad the sliced tensor to the desired shape
    padded_tensor = tf.pad(sliced_tensor, paddings=[[0, padding[0]], [0, padding[1]], [0, padding[2]]])

    return padded_tensor

# Example usage:
t = tf.random.normal((5, 7, 3))  # Example input tensor
h = 4
w = 6
resized_tensor = resize_tensor(t, h, w)
print("Resized tensor shape:", resized_tensor.shape)

InvalidArgumentError: {{function_node __wrapped__Pack_N_3_device_/job:localhost/replica:0/task:0/device:GPU:0}} Shapes of all inputs must match: values[0].shape = [3] != values[1].shape = [] [Op:Pack] name: 

In [2]:
# Basic information about the dataset is here: https://paperswithcode.com/dataset/proteins
# "clean" removes isomorphic (non-distinguishable) graphs
dataset = spektral.datasets.TUDataset('PROTEINS', clean=True)
#dataset = CustomTUDataset('PROTEINS', clean=True)
#dataset = spektral.datasets.citation.Citation.available_datasets()
#New in version 1.2: sparse was renamed to sparse_output
#current version of scikit learn is 1.4.0. I downgraded it to 1.1.3.

Could not read URL https://chrsmrrs.github.io/datasets/docs/datasets/


ValueError: Unknown dataset PROTEINS. See TUDataset.available_datasets() for a complete list ofavailable datasets.

In [ ]:
#pubmed = spektral.datasets.citation.Citation("Pubmed")

In [ ]:
#reddit = spektral.datasets.graphsage.Reddit()

In [ ]:
#https://chrsmrrs.github.io/datasets/docs/datasets/
dataset = spektral.datasets.TUDataset('DD', clean=True)

In [ ]:
#reddit[0]

In [ ]:
# Transforms are inspired by torchvision-like libraries.
# You can apply them using apply (in place) or map (returns a list instead).
dataset.apply(T.ClusteringCoeff())

In [ ]:
# LayerPreprocess allows to apply any preprocessing as suggested by a layer,
# in this case adding self-loops and normalizing the adjacency matrix.
dataset.apply(
    T.LayerPreprocess(layers.GCNConv)
)

In [ ]:
# We split by stratifying on the labels
train_idx, test_idx = train_test_split(range(len(dataset)), stratify = [g.y for g in dataset])
train_dataset = dataset[train_idx]
test_dataset = dataset[test_idx]

In [ ]:
# In alternative, we can use BatchLoader to create "classic" mini-batches by padding
# to the size of the largest graph.
train_loader = spektral.data.loaders.BatchLoader(train_dataset, batch_size=8)
for inputs_alt, labels_alt in train_loader:
  print(inputs_alt[0].shape)
  print(inputs_alt[1].shape)
  break

In [ ]:
class MLP1(tf.keras.layers.Layer):
    def __init__(self, hidden_dim = 32, out_dim = 32):
        super(MLP1, self).__init__()
        self.linear1 = tf.keras.layers.Dense(hidden_dim, activation = "relu")
        self.linear2 = tf.keras.layers.Dense(hidden_dim, activation = "relu")
        self.linear3 = tf.keras.layers.Dense(hidden_dim)

    def call(self, inputs):
        ret = self.linear1(inputs)
        ret = self.linear2(ret)
        ret = self.linear3(ret)
        return ret

In [ ]:
class MLP2(tf.keras.layers.Layer):
    def __init__(self, hidden_dim = 32, out_dim = 32):
        super(MLP2, self).__init__()
        self.linear1 = tf.keras.layers.Dense(hidden_dim, activation = "relu")
        self.linear2 = tf.keras.layers.Dense(hidden_dim, activation = "relu")
        self.linear3 = tf.keras.layers.Dense(hidden_dim)

    def call(self, inputs):
        ret = self.linear1(inputs)
        ret = self.linear2(ret)
        ret = self.linear3(ret)
        return ret

In [ ]:
class nu_log_initializer (tf.keras.initializers.Initializer):
    def __init__ (self):
        self.uniform = tf.keras.initializers.RandomUniform(minval = 0, maxval = 1)

    def __call__(self, shape, dtype = None, **kwargs):
        ret = tf.math.log(tf.math.log(self.uniform(shape))*(-0.5))
        return tf.cast(ret, dtype = tf.complex64)

In [ ]:
class theta_log_initializer (tf.keras.initializers.Initializer):
    def __init__ (self):
        self.uniform = tf.keras.initializers.RandomUniform(minval = 0, maxval = 1)

    def __call__(self, shape, dtype = None, **kwargs):
        ret = tf.math.log(2*math.pi*self.uniform(shape))
        return tf.cast(ret, dtype = tf.complex64)

In [ ]:
class complex_initializer (tf.keras.initializers.Initializer):
    def __init__ (self):
        self.real_uniform = tf.keras.initializers.GlorotUniform()
        self.imag_uniform = tf.keras.initializers.GlorotUniform()

    def __call__(self, shape, dtype = None, **kwargs):
        return tf.dtypes.complex(self.real_uniform(shape)/2,self.imag_uniform(shape)/2)

In [ ]:
def Nk_computing(adj, K):
    K = 5
#for now I will use it like this
#as soon as possible I will put this in preprocessing, to avoid computing this every time
    adjacency_powers = []#in position n,i,j there is a 1 if i and j are at distance n
    #is the powers of the adjacency matrix, but it always has a 1 if a value is positive
    adjacency_powers.append(adj)
    all_previous = [] #I need this to check if there weren't any before ()
    #(I will optimize it later, but right now, something will be computed twice if there are
    #two paths between two nodes with different lengths)
    #at the start it will take the diagonal identity matrix
    all_previous.append(tf.eye(adj.shape[0], dtype=tf.int32))
    for i in range(K):
        to_append = adjacency_powers[-1]@adj
        to_append = tf.cast(to_append>0, tf.int32)
        adjacency_powers.append(to_append)
        to_append_2 = all_previous[-1] + adjacency_powers[-2]
        to_append_2 = tf.cast(to_append_2>0, tf.int32)
        all_previous.append(to_append_2)
    adjacency_powers = tf.convert_to_tensor(adjacency_powers)
    all_previous = tf.convert_to_tensor(all_previous)
    distances = tf.cast(adjacency_powers>all_previous,tf.int32)
    identity = tf.reshape(tf.eye(adj.shape[0],dtype=tf.int32),[1,adj.shape[0],adj.shape[0]])
    to_ADD = tf.concat((identity,distances),0)
    to_ADD = tf.cast(to_ADD, tf.float32)
    return to_ADD
#one = MLP1()
#two = MLP2()
#res = one(nodes)
#broadcasted_mat_res = tf.broadcast_to(mat_res,(7,5,5,32))
#new_thing = tf.reduce_sum(broadcasted_mat_res * broadcasted_to_ADD, axis = 2)
#new_ret = two(new_thing)


In [ ]:

def batch_Nk_computing(adj, K):
    K = 5
    adj = tf.cast(adj>0.0,tf.int32)
    #for now I will use it like this
    #as soon as possible I will put this in preprocessing, to avoid computing this every time
    adjacency_powers = []#in position n,i,j there is a 1 if i and j are at distance n
    #is the powers of the adjacency matrix, but it always has a 1 if a value is positive
    adjacency_powers.append(adj)
    all_previous = [] #I need this to check if there weren't any before ()
    #(I will optimize it later, but right now, something will be computed twice if there are
    #two paths between two nodes with different lengths)
    #at the start it will take the diagonal identity matrix
    #all_previous.append(tf.eye(adj.shape[0], dtype=tf.int32))
    all_previous.append(tf.eye(adj.shape[-1], batch_shape = [adj.shape[0]], dtype=tf.int32))
    for i in range(K):
        to_append = adjacency_powers[-1]@adj
        to_append = tf.cast(to_append>0, tf.int32)
        adjacency_powers.append(to_append)
        to_append_2 = all_previous[-1] + adjacency_powers[-2]
        to_append_2 = tf.cast(to_append_2>0, tf.int32)
        all_previous.append(to_append_2)
    adjacency_powers = tf.convert_to_tensor(adjacency_powers)
    all_previous = tf.convert_to_tensor(all_previous)
    distances = tf.cast(adjacency_powers>all_previous,tf.int32)
    identity = tf.eye(adj.shape[-1],batch_shape = [1,adj.shape[0]],dtype=tf.int32)
    #tf.broadcast_to(identity, (adj.shape[0],1,adj.shape[-1], adj.shape[-1]))
    to_ADD = tf.concat((identity,distances),0)
    to_ADD = tf.cast(to_ADD, tf.float32)
    return to_ADD
#one = MLP1()
#two = MLP2()
#res = one(nodes)
#broadcasted_mat_res = tf.broadcast_to(mat_res,(7,5,5,32))
#new_thing = tf.reduce_sum(broadcasted_mat_res * broadcasted_to_ADD, axis = 2)
#new_ret = two(new_thing)


In [ ]:
def ADD(mlp1, mlp2, nodes, to_add):
    ret = mlp1(nodes)
    reshaped_to_add = tf.reshape(to_add, to_add.shape+[1])
    broadcasted_to_add = tf.broadcast_to(reshaped_to_add,to_add.shape+[nodes.shape[-1]])
    broadcasted_nodes = tf.broadcast_to(nodes,
                                       (to_add.shape[0],nodes.shape[0],nodes.shape[1],nodes.shape[2]))
    broadcasted_nodes = tf.reshape(broadcasted_nodes,
                                  (to_add.shape[0],nodes.shape[0],nodes.shape[1],1,nodes.shape[2]))
    broadcasted_nodes = tf.broadcast_to(broadcasted_nodes,(to_add.shape[0],nodes.shape[0],nodes.shape[1],nodes.shape[1],nodes.shape[2]))
    ret = tf.reduce_sum(broadcasted_to_add * broadcasted_nodes, axis = 3)
    #check if axis = 3 is right. it might be axis = 2
    ret = mlp2(ret)
    return ret

In [ ]:
class my_GRED_layer(tf.keras.layers.Layer):
    def __init__(self, out_dimension = 32, K = 5, x_dim = 32, state_dim = 32):
        super(my_GRED_layer, self).__init__()
        self.out_dimension = out_dimension
        self.K = K
        self.x_dim = x_dim
        self.state_dim = state_dim
        self.linear1 = MLP1()
        self.linear2 = MLP2()
        self.linear3 = MLP2()
        self.log_nu = self.add_weight(shape = [state_dim], initializer = nu_log_initializer(), dtype = tf.complex64)
        self.log_theta =self.add_weight(shape = [state_dim], initializer = theta_log_initializer(), dtype = tf.complex64)
        self.W_in = self.add_weight(shape = (state_dim,x_dim), initializer = complex_initializer(), dtype = tf.complex64)
        self.W_out = self.add_weight(shape = (x_dim, state_dim), initializer = complex_initializer(), dtype = tf.complex64)

    def call(self, inputs):
        h,a = inputs
        to_ADD = batch_Nk_computing(a, self.K)
        #for now I will compute everything on the spot. Later I will preprocess this
        xs = ADD(self.linear1, self.linear2, h, to_ADD)
        lambda_1 = tf.math.exp((-1)*tf.math.exp(self.log_nu) + 1j*tf.math.exp(self.log_theta))
        #optimal multiplication by diagonal matrix is  matrix * tf.transpose([vector]) if the diagonal matrix is on the left
        #matrix * tf.transpose([vector]) == tf.linalg.diag(vector)@matrix
        #matrix*vector == matrix@tf.linalg.diag(vector)
        broadcasted_lambda_1 = tf.broadcast_to(lambda_1, (xs.shape[0],xs.shape[1], self.state_dim))
        #I will probably need to cast something to complex values
        broadcasted_lambda_1 = tf.broadcast_to(lambda_1, (xs.shape[0],xs.shape[1],self.state_dim))
        broadcasted_w_in = tf.broadcast_to(self.W_in, (xs.shape[0],xs.shape[1],self.state_dim,self.x_dim))#i can optimize this by adding the batch dimension later
        powers = tf.transpose(tf.broadcast_to(tf.range(xs.shape[0], dtype = tf.float32),(self.state_dim,xs.shape[1],xs.shape[0])),[2,1,0])
        powers = tf.cast(powers, dtype = tf.complex64)
        lambda_powers = broadcasted_lambda_1 ** powers
        transposed_lambda_powers = tf.transpose(tf.reshape(lambda_powers,[xs.shape[0],xs.shape[1],1,self.state_dim]),[0,1,3,2])
        lambda_times_w_in = transposed_lambda_powers * broadcasted_w_in
        #(tf.transpose([lambda_1])*w_in)@tf.reshape(xs[0,0,0],[32,1])
        xs = tf.cast(xs, dtype = tf.complex64)
        broadcasted_lambda_times_w_in = tf.broadcast_to(
            tf.reshape(lambda_times_w_in,[xs.shape[0],xs.shape[1],1,self.state_dim,self.x_dim]),
            [xs.shape[0],xs.shape[1],xs.shape[2],self.state_dim,self.x_dim])
        s_K = broadcasted_lambda_times_w_in @ tf.reshape(xs,
                                                         [xs.shape[0],xs.shape[1],xs.shape[2],self.x_dim,1])
        s_K = tf.reduce_sum(tf.reshape(s_K,[xs.shape[0],xs.shape[1],xs.shape[2],self.state_dim]),0)
        #now is just ,multiply by w_out, take the real part and put in the last MLP
        broadcasted_w_out = tf.broadcast_to(self.W_out, [xs.shape[1],xs.shape[2],self.x_dim,self.state_dim])
        reshaped_s_K = tf.reshape(s_K,[xs.shape[1],xs.shape[2],self.state_dim,1])
        output = tf.reshape(broadcasted_w_out@reshaped_s_K,[xs.shape[1],xs.shape[2],self.x_dim])
        output = tf.math.real(output)
        output = self.linear3(output)
        return output

In [ ]:
class my_GRED_model(tf.keras.Model):
    def __init__(self, out_dim = 2):
        super(my_GRED_model, self).__init__()
        self.out_dim = out_dim
        self.gred_1 = my_GRED_layer()
        self.out_linear = tf.keras.layers.Dense(out_dim)
        

    def call(self, inputs):
        ret = self.gred_1(inputs)
        ret = tf.reduce_mean(ret,1)
        ret = self.out_linear(ret)
        return ret

In [ ]:
mod = my_GRED_model()
mod.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

In [ ]:
#mod.fit(train_loader, epochs=10, steps_per_epoch=train_loader.steps_per_epoch)

In [ ]:
sgd = tf.keras.optimizers.experimental.SGD()

In [ ]:
#@tf.function(reduce_retracing=True)
def train_step(batch):
  
        xb, yb = batch
        with tf.GradientTape() as tape:
            # Get the predictions of the model
            y_predicted = mod(xb)

            # Compute the average loss of the predictions
            ce = tf.keras.losses.CategoricalCrossentropy()(yb, y_predicted)
            loss = ce

        # Get the gradients of the parameters
        grads = tape.gradient(loss, mod.trainable_variables)

        # Update the parameters using gradient descent
        sgd.apply_gradients(zip(grads, mod.trainable_variables))

        return ce

In [ ]:
epochs = 5
for i in range(epochs):
    count = 0
    for batch in train_loader:
        if count >= train_loader.steps_per_epoch:
            break
        train_step(batch)
        count+=1
        if count == 90:
            print(count)
    print(i)
    

In [ ]:
#I have no clue why the outputs are the opposite of what it should be, check the loss function
#also, every time it runs, since there are different tensor shapes, retracing makes a mess.
#reduce_retracing = True should do exactly this, but it will see exactly how to do it
#also, check why fit does not work and check that all calculations are correct in the other one
#especially when computing the last probability
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#this should turn off the warnings

In [ ]:
a = tf.keras.activations.softmax((-1)*mod(batch[0]))

In [ ]:
a

In [ ]:
batch[1]

In [ ]:
tf.reduce_mean(tf.cast((a[:,0]>a[:,1]) == (batch[1][:,0] > batch[1][:,1]),tf.float32))

In [ ]:
tf.keras.losses.BinaryCrossentropy()((-1)*mod(batch[0]),batch[1])

In [ ]:
tf.keras.metrics.Accuracy()((-1)*mod(batch[0]),batch[1])

In [ ]:
a = (-1)*mod(batch[0])

In [ ]:
tf.keras.metrics.Accuracy()(a,batch[1])

In [ ]:
xs = tf.random.uniform((7,8,12,32))
w_in = tf.random.uniform((64,32))
w_out = tf.random.uniform((32,64))
lambda_1 = tf.random.uniform([64])
broadcasted_lambda_1 = tf.broadcast_to(lambda_1, (7,8,64))
broadcasted_w_in = tf.broadcast_to(w_in, (7,8,64,32))#i can optimize this by adding the batch dimension later
powers = tf.transpose(tf.broadcast_to(tf.range(7, dtype = tf.float32),(64,8,7)),[2,1,0])
lambda_powers = broadcasted_lambda_1 ** powers
transposed_lambda_powers = tf.transpose(tf.reshape(lambda_powers,[7,8,1,64]),[0,1,3,2])
lambda_times_w_in = transposed_lambda_powers * broadcasted_w_in
#(tf.transpose([lambda_1])*w_in)@tf.reshape(xs[0,0,0],[32,1])

broadcasted_lambda_times_w_in = tf.broadcast_to(tf.reshape(lambda_times_w_in,[7,8,1,64,32]),[7,8,12,64,32])
#tf.reshape(broadcasted_lambda_times_w_in @ tf.reshape(xs,[7,8,12,32,1]),[7,8,12,64])[1,0]
#tf.transpose((tf.transpose([lambda_1])*w_in)@tf.reshape(xs[0,0,0],[32,1]))
s_K = broadcasted_lambda_times_w_in @ tf.reshape(xs,[7,8,12,32,1])
s_K = tf.reduce_sum(tf.reshape(s_K,[7,8,12,64]),0)
#now is just ,multiply by w_out, take the real part and put in the last MLP
s_K.shape
broadcasted_w_out = tf.broadcast_to(w_out, [8,12,32,64])
reshaped_s_K = tf.reshape(s_K,[8,12,64,1])
output = tf.reshape(broadcasted_w_out@reshaped_s_K,[8,12,32])
output = tf.math.real(output)
output = MLP2()(output)

In [ ]:
mod(batch[0])

In [ ]:
batch[1]